In [109]:
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import numpy as np
import os
from reload_recursive import reload_recursive
import helpers
import pyperclip
import statsmodels.api as sm
import subprocess

from mri_data import file_manager as fm
from mri_data import utils

In [110]:
drive_root = fm.get_drive_root()
msmri_home = Path("/home/srs-9/Projects/ms_mri")
msmri_datadir = msmri_home / "data"
curr_dir = Path("/home/srs-9/Projects/ms_mri/analysis/paper1")
data_dir = curr_dir / "data0"
showfigs = False
# pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.options.display.precision = 3

In [111]:
df1 = pd.read_csv(data_dir / "t1_data_full.csv")
df1.set_index('subid', inplace=True)
df1 = helpers.set_dz_type5(df1)

df2 = pd.read_csv(data_dir / "t1_2_data_full.csv")
df2.set_index('subid', inplace=True)
df2 = helpers.set_dz_type5(df2)

df3 = pd.read_csv(data_dir / "t1_3_data_full.csv")
df3.set_index('subid', inplace=True)
df3 = helpers.set_dz_type5(df3)

df_flair = pd.read_csv(data_dir / "flair_data_full.csv")
df_flair.set_index('subid', inplace=True)
df_flair = helpers.set_dz_type5(df_flair)

df_t1_flair = pd.read_csv(data_dir / "t1_flair_data_full.csv")
df_t1_flair.set_index('subid', inplace=True)
df_t1_flair = helpers.set_dz_type5(df_t1_flair)

In [180]:
df = pd.DataFrame(index=df1.index)
df['age'] = df1['age']
df['dz_type5'] = df1['dz_type5']
df['EDSS'] = df1['extracted_EDSS']
df['choroid_volume1'] = df1['choroid_volume']
df['choroid_volume2'] = df2['choroid_volume']
df['cp_diff'] = df['choroid_volume1'] - df['choroid_volume2']
df['choroid_volume_fl'] = df_flair['choroid_volume']
df['choroid_volume_t1fl'] = df_t1_flair['choroid_volume']


In [181]:
df = df.sort_values(by="cp_diff", ascending=False)
df_pms = df[df['dz_type5']=="PMS"]
df_rms = df[df['dz_type5']=="RMS"]
df_pms_w = df[(df['dz_type5']=="PMS") & (df1['flair_contrast'] == "WITH")]

/tmp/ipykernel_529437/2774009775.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_pms_w = df[(df['dz_type5']=="PMS") & (df1['flair_contrast'] == "WITH")]


In [190]:
model_data = df2.copy()
# model_data.loc[model_data['dz_type5']=="PMS", "choroid_volume"] = df.loc[df['dz_type5']=="PMS", "choroid_volume1"]
# subs_to_change = [1453, 1276, 1370, 1049, 1077, 1299, 1155, 2027, 1322]
subs_to_change = [1530, 1435, 1442, 1106, 1068, 2051, 1052, 1217, 1049, 1508]
model_data.loc[subs_to_change, "choroid_volume"] = df.loc[subs_to_change, "choroid_volume1"]

model_data = helpers.fix_edss(model_data)
model_data = helpers.clean_df(model_data)

model_data = model_data[model_data['dz_type5'].isin(["RMS", "PMS"])]
model_data = pd.concat((model_data, pd.get_dummies(model_data["sex"], dtype="int")), axis=1)

model_data['EDSS'] = model_data['EDSS'].astype("float")

In [191]:
res = sm.OLS.from_formula("EDSS ~ choroid_volume + tiv + age + Female", data=model_data).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   EDSS   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.197
Method:                 Least Squares   F-statistic:                     25.24
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           1.28e-18
Time:                        11:37:02   Log-Likelihood:                -785.53
No. Observations:                 397   AIC:                             1581.
Df Residuals:                     392   BIC:                             1601.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9414      1.429      0.

In [192]:
model_data = df1.copy()
model_data = helpers.fix_edss(model_data)
model_data = helpers.clean_df(model_data)

model_data = model_data[model_data['dz_type5'].isin(["RMS", "PMS"])]
model_data = pd.concat((model_data, pd.get_dummies(model_data["sex"], dtype="int")), axis=1)
model_data['EDSS'] = model_data['EDSS'].astype("float")
res = sm.OLS.from_formula("EDSS ~ choroid_volume + tiv + age + Female", data=model_data).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   EDSS   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.201
Method:                 Least Squares   F-statistic:                     25.94
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           4.31e-19
Time:                        11:37:33   Log-Likelihood:                -784.41
No. Observations:                 397   AIC:                             1579.
Df Residuals:                     392   BIC:                             1599.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          0.9531      1.423      0.

In [210]:
check = model_data.loc[model_data['EDSS'].dropna().index, :]

In [ ]:
influence = res.get_influence()
cooks_d = influence.cooks_distance
cooks_d[1]

In [206]:
influence.exog.shape

(397, 5)

In [161]:
sub = 1242

dataroot = drive_root / "3Tpioneer_bids"
labelroot = drive_root / "srs-9/3Tpioneer_bids_predictions"
label1 = labelroot / df1.loc[sub, 'sub-ses'] / df1.loc[sub, 'label']
label2 = labelroot / df2.loc[sub, 'sub-ses'] / df2.loc[sub, 'label']
label_flair = labelroot / df_flair.loc[sub, 'sub-ses'] / df_flair.loc[sub, 'label']
label_t1_flair = labelroot / df_t1_flair.loc[sub, 'sub-ses'] / df_t1_flair.loc[sub, 'label']

t1 = dataroot / df1.loc[sub, 'sub-ses'] / "t1.nii.gz"
t1_gd = dataroot / df1.loc[sub, 'sub-ses'] / "t1_gd.nii.gz"

flair = dataroot / df1.loc[sub, 'sub-ses'] / "flair.nii.gz"
t1_gd = dataroot / df1.loc[sub, 'sub-ses'] / "t1_gd.nii.gz"

cmd = utils.open_itksnap_workspace_cmd(images=[t1, t1_gd, flair], 
                                       labels=[label1, label2, label_flair, label_t1_flair], 
                                       win=True)
pyperclip.copy(cmd)
print(cmd)
# subprocess.Popen(cmd.split(" "))

itksnap -g H:/3Tpioneer_bids/sub-ms1242/ses-20170830/t1.nii.gz -o H:/3Tpioneer_bids/sub-ms1242/ses-20170830/t1_gd.nii.gz H:/3Tpioneer_bids/sub-ms1242/ses-20170830/flair.nii.gz -s H:/srs-9/3Tpioneer_bids_predictions/sub-ms1242/ses-20170830/t1_choroid_pineal_pituitary_T1-1_pred.nii.gz H:/srs-9/3Tpioneer_bids_predictions/sub-ms1242/ses-20170830/t1_choroid_pineal2_pituitary_T1-1_pred.nii.gz H:/srs-9/3Tpioneer_bids_predictions/sub-ms1242/ses-20170830/flair_choroid_pineal_pituitary_FLAIR-1_pred.nii.gz H:/srs-9/3Tpioneer_bids_predictions/sub-ms1242/ses-20170830/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


In [157]:
t1_gd.exists()

False

In [148]:
sub = 1235
t1 = dataroot / df1.loc[sub, 'sub-ses'] / "proc/t1_std.nii.gz"
flair = dataroot / df1.loc[sub, 'sub-ses'] / "proc/flair-brain-mni_reg.nii.gz"
cmd = utils.open_itksnap_workspace_cmd(images=[t1, flair])
subprocess.Popen(cmd.split(" "))

<Popen: returncode: None args: ['itksnap', '-g', '/mnt/h/3Tpioneer_bids/sub-...>

In [156]:
df1.loc[1491, 'flair_contrast']

'WITH'

In [18]:
print(t1)

/media/smbshare/3Tpioneer_bids/sub-ms1370/ses-20210421/proc/t1_std.nii.gz


In [ ]:
fig, axs = plt.subplots(2,1,figsize=(12, 4))

axs[0].hist(df1.loc[df1['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])
axs[1].hist(df3.loc[df2['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])

In [ ]:
plt.scatter(df1.loc[df1['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'], 
            df3.loc[df3['dz_type5'].isin(["RMS", "PMS"]), 'choroid_volume'])

In [ ]:
plt.hist((df1.loc[df1['dz_type5'].isin(["PMS"]), 'choroid_volume'] - df2.loc[df2['dz_type5'].isin(["PMS"]), 'choroid_volume']))